In [36]:
from pickle import TRUE
import re

# Function to convert an address to a binary string (32-bit)
def address_to_binary(address):
    return format(address, '032b')  # Converts address to 32-bit binary string

# Function to convert a register name (e.g., $t0, $a1) to its binary value
def register_to_binary(reg):
    reg_map = {
        "$zero": "00000", "$t0": "01000", "$t1": "01001", "$t2": "01010", "$t3": "01011", "$t4": "01100",
        "$t5": "01101", "$t6": "01110", "$t7": "01111", "$s0": "10000", "$s1": "10001", "$s2": "10010",
        "$s3": "10011", "$s4": "10100", "$s5": "10101", "$s6": "10110", "$s7": "10111", "$a0": "00100",
        "$a1": "00101", "$a2": "00110", "$a3": "00111", "$v0": "00010", "$v1": "00011", "$sp": "11101",
        "$1": "00001"  # Register $1 used for storing addresses
    }
    return reg_map.get(reg, "00000")  # Default to "00000" for unknown registers

# Function to convert R-type instructions to binary
def r_type_to_binary(opcode, rd, rs, rt):
    funct_map = {
        "add": "100000", "sub": "100010", "and": "100100", "or": "100101", "slt": "101010"
    }
    funct = funct_map[opcode]
    shamt = "00000"  # shamt is always 0 for these instructions
    return f"000000{register_to_binary(rs)}{register_to_binary(rt)}{register_to_binary(rd)}{shamt}{funct}"

# Function to convert I-type instructions to binary
def i_type_to_binary(opcode, rt, rs, imm):
   if imm < 0:
       imm_binary = format(imm & 0xFFFF, '016b')  # 16-bit signed immediate
   else:
       imm_binary = format(imm & 0xFFFF, '016b')  # 16-bit signed immediate
   opcode_binary_map = {
        "lw": "100011", "addi": "001000", "beq": "000100", "bne": "000101"
    }
   opcode_binary = opcode_binary_map[opcode]
   return f"{opcode_binary}{register_to_binary(rs)}{register_to_binary(rt)}{imm_binary}"

# Function to convert J-type instructions to binary
def j_type_to_binary(opcode, address_str, label_map):
    if address_str in label_map:
        address = label_map[address_str]//4
    else:
        try:
            address = int(address_str)
        except ValueError:
            print(f"Invalid jump address: {address_str}")
            return ""
    opcode_binary = "000010"  # Opcode for j
    return f"{opcode_binary}{format(address, '026b')}"


# Function to parse and convert a MIPS instruction to binary
def instruction_to_binary(instruction, memory_address_map, label_map, current_address):
    parts = re.split(r'[,\s()]+', instruction.strip())  # Split by space, comma, or parenthesis
    opcode = parts[0]

    # Handle R-type instructions
    if opcode in {"add", "sub", "and", "or", "slt"}:
        rd, rs, rt = parts[1], parts[2], parts[3]
        return r_type_to_binary(opcode, rd, rs, rt)

    # Handle I-type instructions (lw, addi, beq)
    elif opcode == "lw":
        rt, var = parts[1], parts[2].strip()  # Strip any whitespace
        rs = "$1"  # Assume base address is stored in $1 for "lw"
        if var in memory_address_map:
            imm = memory_address_map[var]
            return i_type_to_binary(opcode, rt, rs, imm)
        else:
            print(f"Error: '{var}' not found in memory_address_map!")
            return ""

    elif opcode == "addi":
        rt, rs, imm_str = parts[1], parts[2], parts[3]
        if imm_str.startswith('-'):
            imm = int(imm_str)  # Negative immediate value for addi
        else:
            imm = int(imm_str)  # Immediate value for addi
        return i_type_to_binary(opcode, rt, rs, imm)


    elif opcode == "beq":
        rs, rt, label = parts[1], parts[2], parts[3]
        if label in label_map:
            target_address = label_map[label]
            offset = (target_address - (current_address + 4)) // 4  # Relative address for branch
            return i_type_to_binary(opcode, rt, rs, offset)
        else:
            print(f"Error: label '{label}' not found in label_map!")
            return ""

    elif opcode == "bne":
        rs, rt, label = parts[1], parts[2], parts[3]
        if label in label_map:
            target_address = label_map[label]
            offset = (target_address - (current_address + 4)) // 4  # Relative address for branch
            return i_type_to_binary(opcode, rt, rs, offset)
        else:
            print(f"Error: label '{label}' not found in label_map!")
            return ""

    # Handle J-type instructions (j)
    elif opcode == "j":
        address = parts[1]
        return j_type_to_binary(opcode, address, label_map)


    return ""  # Return empty string for unsupported instructions

# Function to allocate memory for variables in the .data section
def allocate_data_memory(data_map):
    memory = {}  # Simulated memory (dictionary)
    memory_address_map = {}  # Maps variable name to memory address
    base_address = 0
    for var, value in data_map.items():
        memory[base_address] = value  # Store value in memory
        memory_address_map[var] = base_address  # Map variable to its base address
        base_address += 4  # Word-aligned addresses
    return memory, memory_address_map

# Function to parse the .data section
def parse_data_section(data_section):
    data_map = {}
    for line in data_section.splitlines():
        line = line.strip()
        if not line or ':' not in line:
            continue  # Skip empty lines or invalid data lines
        variable, definition = line.split(':')
        variable = variable.strip()
        definition = definition.strip()
        if ".word" in definition:
            value = int(definition.split()[1])  # Extract the value after `.word`
            data_map[variable] = value
    return data_map

# Function to parse the .text section and convert to binary
def parse_text_section(text_section, memory_address_map):
    binary_instructions = []
    current_address = 0
    label_map = {}

    # First pass: Identify labels and their addresses
    for line in text_section.strip().splitlines():
        if line.endswith(':'):
            label = line[:-1].strip()  # Remove ':' and get label name
            label_map[label] = current_address
        else:
            current_address += 4  # Increment address for each instruction

    current_address = 0  # Reset for the second pass
    # Second pass: Convert instructions to binary
    for line in text_section.strip().splitlines():
        if line.endswith(':'):
            continue  # Skip labels
        binary_instruction = instruction_to_binary(line, memory_address_map, label_map, current_address)
        if binary_instruction:
            binary_instructions.append(binary_instruction)
        current_address += 4  # Each instruction is 4 bytes apart

    return binary_instructions


class MIPSExecution:

    def __init__(self, instructions, memory, registers):
        self.instructions = instructions  # Binary instructions
        self.memory = memory  # Memory dictionary
        self.registers = registers  # Register dictionary
        self.pc = 0  # Program counter
        self.running = True

    def execute_instruction(self, instruction):
        print(f"Executing instruction at PC={self.pc}: {instruction}")

        opcode = instruction[:6]  # Get the opcode (first 6 bits)

        if opcode == '100011':  # lw
            rs = int(instruction[6:11], 2)  # Base register
            rt = int(instruction[11:16], 2)  # Destination register
            offset = int(instruction[16:], 2)  # Offset (immediate value)
            address = self.registers[rs] + offset  # Calculate effective address using base register and offset
            self.registers[rt] = self.memory.get(address, 0)  # Load word from memory into destination register


        elif opcode == '001000':  # addi
          rt = int(instruction[11:16], 2)  # Destination register
          rs = int(instruction[6:11], 2)  # Source register
          immediate = int(instruction[16:], 2)  # Immediate value
          if immediate & 0x8000:  # Check if the immediate value is negative
              immediate = -((~immediate + 1) & 0xFFFF)  # Sign-extend the 16-bit immediate value
          self.registers[rt] = self.registers[rs] + immediate  # Add immediate to register

        elif opcode == '000100':  # beq
          rs = int(instruction[6:11], 2)  # Source register
          rt = int(instruction[11:16], 2)  # Target register
          offset = int(instruction[16:], 2)  # Branch offset
          if offset & 0x8000:  # Check if the offset is negative
              offset = -((~offset + 1) & 0xFFFF)  # Sign-extend the 16-bit offset
          else:
              offset = offset  # Positive offset
          if self.registers[rs] == self.registers[rt]:
              self.pc += offset  # Update PC for branch
              return  # Don't increment PC for branch and execute the instruction at the target address
          else:
              # Execute the instructions after the branch
              self.pc += 0  # Increment PC to execute the next instruction


        elif opcode == '000101':  # bne
          rs = int(instruction[6:11], 2)  # Source register
          rt = int(instruction[11:16], 2)  # Target register
          offset = int(instruction[16:], 2)  # Branch offset
          if offset & 0x8000:  # Check if the offset is negative
              offset = -((~offset + 1) & 0xFFFF)  # Sign-extend the 16-bit offset
          else:
              offset = offset  # Positive offset
          if self.registers[rs] != self.registers[rt]:
              self.pc += offset  # Update PC for branch
              return  # Don't increment PC for branch and execute the instruction at the target address
          else:
              # Execute the instructions after the branch
              self.pc += 0  # Increment PC to execute the next instruction



        elif opcode == '000000':  # R-type (add)
            funct = instruction[26:]  # Get the function code (last 6 bits)
            rs = int(instruction[6:11], 2)  # Source register
            rt = int(instruction[11:16], 2)  # Second source register
            rd = int(instruction[16:21], 2)  # Destination register

            if funct == '100000':  # add
                self.registers[rd] = self.registers[rs] + self.registers[rt]  # Add
            elif funct == '100010':  # sub
                self.registers[rd] = self.registers[rs] - self.registers[rt]  # Subtract
            elif funct == '100100':  # and
                self.registers[rd] = self.registers[rs] & self.registers[rt]  # Bitwise AND
            elif funct == '100101':  # or
                self.registers[rd] = self.registers[rs] | self.registers[rt]  # Bitwise OR
            elif funct == '101010':  # slt
                self.registers[rd] = 1 if self.registers[rs] < self.registers[rt] else 0  # Set on less than
            else:
                print(f"Unknown R-type function: {funct}")

        elif opcode == '000010':  # j (jump)
            address = int(instruction[6:], 2)  # Jump target address
            self.pc = address  # Set PC to jump target address
            return TRUE # Don't increment PC after jump


        else:
            print(f"Unknown instruction: {instruction}")

        # Rest of the code...
        print(f"Registers after execution: {self.registers}")
        print(f"Memory state: {self.memory}\n")



    def run(self):
      #  while self.running and self.pc < len(self.instructions):
      #       instruction = self.instructions[self.pc]
      #       self.execute_instruction(instruction)
      #       self.pc += 1  # Move to the next instruction, unless branch or jump was taken
      while self.running and self.pc < len(self.instructions):
        instruction = self.instructions[self.pc]
        jumped = self.execute_instruction(instruction)
        if not jumped:
            self.pc += 1


    def print_state(self):
        print("Registers:")
        for i in range(32):
            print(f"${i}: {self.registers[i]}")
        print("\nMemory:")
        for address, value in self.memory.items():
            print(f"Address {address}: {value}")


def run_mips_simulator(data_section, text_section):
    data_map = parse_data_section(data_section)  # Parse data section
    memory, memory_address_map = allocate_data_memory(data_map)  # Allocate memory
    instructions = parse_text_section(text_section, memory_address_map)  # Convert instructions to binary
    registers = {i: 0 for i in range(32)}  # Initialize registers to 0

    simulator = MIPSExecution(instructions, memory, registers)  # Create simulator
    simulator.run()  # Execute the instructions

# Main function to run the simulator
# Main program
def main():
    input_file = "mips.txt"  # Replace with your input file path
    output_file = "output.txt"  # Output file for binary instructions

    # Read the file and split it into .data and .text sections
    with open(input_file, "r") as file:
        content = file.read()

    data_section = ""
    text_section = ""
    in_text_section = False

    for line in content.splitlines():
        line = line.strip()
        if line == ".data":
            in_text_section = False
        elif line == ".text":
            in_text_section = True
        elif in_text_section:
            text_section += line + "\n"
        else:
            data_section += line + "\n"

    # Parse the .data section and allocate memory
    data_map = parse_data_section(data_section)  # Parse data section
    memory, memory_address_map = allocate_data_memory(data_map)  # Allocate memory

    # Parse the .text section and get binary instructions
    binary_instructions = parse_text_section(text_section, memory_address_map)

    # Save the binary instructions to an output file
    with open(output_file, "w") as out_file:
        for instruction in binary_instructions:
            out_file.write(instruction + "\n")

    # Initialize registers to 0
    registers = {i: 0 for i in range(32)}

    # Create and run the simulator
    simulator = MIPSExecution(binary_instructions, memory, registers)
    simulator.run()  # Run the simulation

    # Print final state (registers and memory)
    simulator.print_state()


if __name__ == "__main__":
    main()


Executing instruction at PC=0: 10001100001010000000000000000000
Registers after execution: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 2, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0}
Memory state: {0: 2, 4: 3}

Executing instruction at PC=1: 10001100001010010000000000000100
Registers after execution: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 2, 9: 3, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0}
Memory state: {0: 2, 4: 3}

Executing instruction at PC=2: 00000001000010010101000000101010
Registers after execution: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 2, 9: 3, 10: 1, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0}
Memory state: {0: 2, 4: 3}